In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

In [10]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Dataset") \
    .getOrCreate()

In [21]:
path = "ratings.csv"
df = spark.read.csv(path, header=True)
df.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
path2 = "movies.csv"
df2 = spark.read.csv(path2, header=True)
df2 = df2.withColumnRenamed("movieId", "movieId_movies")
df2.show(5)

+--------------+--------------------+--------------------+
|movieId_movies|               title|              genres|
+--------------+--------------------+--------------------+
|             1|    Toy Story (1995)|Adventure|Animati...|
|             2|      Jumanji (1995)|Adventure|Childre...|
|             3|Grumpier Old Men ...|      Comedy|Romance|
|             4|Waiting to Exhale...|Comedy|Drama|Romance|
|             5|Father of the Bri...|              Comedy|
+--------------+--------------------+--------------------+
only showing top 5 rows



In [13]:
df = df.join(df2, df.movieId == df2.movieId_movies, 'left')
df.show(truncate=False)

+------+-------+------+----------+--------------+------------------------------------------------------------------+-------------------------------+
|userId|movieId|rating|timestamp |movieId_movies|title                                                             |genres                         |
+------+-------+------+----------+--------------+------------------------------------------------------------------+-------------------------------+
|1     |296    |5.0   |1147880044|296           |Pulp Fiction (1994)                                               |Comedy|Crime|Drama|Thriller    |
|1     |306    |3.5   |1147868817|306           |Three Colors: Red (Trois couleurs: Rouge) (1994)                  |Drama                          |
|1     |307    |5.0   |1147868828|307           |Three Colors: Blue (Trois couleurs: Bleu) (1993)                  |Drama                          |
|1     |665    |5.0   |1147878820|665           |Underground (1995)                                       

In [14]:
df = df.drop('movieId_movies')

In [15]:
df = df.withColumn("genres", split(df.genres, r"\|"))

In [16]:
df = df.withColumn("genres", explode(df.genres).alias("genre"))
df.show()

+------+-------+------+----------+--------------------+--------+
|userId|movieId|rating| timestamp|               title|  genres|
+------+-------+------+----------+--------------------+--------+
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|  Comedy|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|   Crime|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|   Drama|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|Thriller|
|     1|    306|   3.5|1147868817|Three Colors: Red...|   Drama|
|     1|    307|   5.0|1147868828|Three Colors: Blu...|   Drama|
|     1|    665|   5.0|1147878820|  Underground (1995)|  Comedy|
|     1|    665|   5.0|1147878820|  Underground (1995)|   Drama|
|     1|    665|   5.0|1147878820|  Underground (1995)|     War|
|     1|    899|   3.5|1147868510|Singin' in the Ra...|  Comedy|
|     1|    899|   3.5|1147868510|Singin' in the Ra...| Musical|
|     1|    899|   3.5|1147868510|Singin' in the Ra...| Romance|
|     1|   1088|   4.0|11

In [17]:
df.filter(df.rating.isNull()).show(10)

+------+-------+------+---------+-----+------+
|userId|movieId|rating|timestamp|title|genres|
+------+-------+------+---------+-----+------+
+------+-------+------+---------+-----+------+



In [18]:
df = df.select("userId", "movieId", "rating", "title", "genres")
df.show(5)

+------+-------+------+--------------------+--------+
|userId|movieId|rating|               title|  genres|
+------+-------+------+--------------------+--------+
|     1|    296|   5.0| Pulp Fiction (1994)|  Comedy|
|     1|    296|   5.0| Pulp Fiction (1994)|   Crime|
|     1|    296|   5.0| Pulp Fiction (1994)|   Drama|
|     1|    296|   5.0| Pulp Fiction (1994)|Thriller|
|     1|    306|   3.5|Three Colors: Red...|   Drama|
+------+-------+------+--------------------+--------+
only showing top 5 rows



In [19]:
# Verificando a contagem de linhas
print(f"Contagem de linhas antes de salvar: {df.count()}")

# Verificando o esquema do DataFrame
df.printSchema()

Contagem de linhas antes de salvar: 67809886
root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = false)



In [20]:
# Salvando o resultado em um CSV
df.write.csv("movies_ratings", header=True, mode='overwrite')